# Twitter download code


In [45]:
#####################################
## Author : Pandramishi Naga Sirisha
## Created on: 07-04-2020
## Last updated on : 07-04-2020
## Last changed purpose: To get racist tweets 
###################################################

### Algorithm:

1. Read the config file
2. Read and validate the configurations
3. Authenticate the Twitter API user
4. Run Steps 5 to end, if downloaded tweets <= maxTweets
5. Use Twitter API call and Tweepy's Cursor to search for posts with #coronavirus. We can filter out retweets, set language in the search.
6. From the returned tweets data in that particular API call, we will extract:

    - text - tweet.full_text
    - date created - tweet.created_at
    - location - tweet.user.location
    
7. Convert the output of (5.) into a pandas dataframe 
8. Clean and preprocess the dataframe. For example, the month and day of the post should be in its own column and the location should be by country.
9. Write/append the dataframe to a csv file.
10. Calculate the remaining tweets to be downloaded, the minid (oldest id of the tweet in the previous API call) and write it back to config file
11. Check if TIME_OUT (twitter limit of 150 API calls for 15mins) is reached:
    - If yes, sleep for 15 mins
    - Else, continue to step . 5
12. Return to (4.) to conduct more searchs if necessary.

In [46]:
import configparser
import tweepy
import csv
import pandas as pd
import datetime
import time

In [47]:
# Read the config file path
config_file = input("Please enter the path to the config file: ")
# 'config_ini_trends.ini'
print(config_file)

Please enter the path to the config file: config_ini_trends.ini
config_ini_trends.ini


In [48]:
# Initialize configparser object and read config file
config = configparser.ConfigParser()

try:
    config.read(config_file)
except:
    print("Config file cannot be read, please check the path")
    
# Read the configurations
output_file = config.get('DEFAULT','output_file_csv').replace('"', '')
tweetCount = int(config.get('DEFAULT','tweetCount'))
maxTweets = int(config.get('DEFAULT','maxTweets'))
minid = int(config.get('DEFAULT','minid'))
area = config.get('DEFAULT','area')

In [49]:
# location 

In [50]:
maxTweets

3000

In [51]:
# Basic sanity checks
def sanity_checks():
    '''This function performs validation checks on the configurations given by user '''
    
    if any([not(x) for x in [output_file,tweetCount,maxTweets]]) == True:
        print("Configurations should not be empty ")
    
    if maxTweets <= tweetCount:
        print("The maxtweets should be greater than tweetCounts")
    
    else:
        print("Sanity checks passed..")
    return
    exit(0)

In [52]:
def authenticate():
    ''' This functions establishes connections to the twitter api'''
    ## Millie's credentials
    consumer_key = 'sjXdAyyk97GcF6cszuV4mRMyw'
    consumer_secret = 'qNRW9dsRaOCf68DUbR6ZjFpHDVFj5pw0FhbOxuq0wcYpgSwhQR'
    access_token = '1232562557778448385-mtvjM6c557WT0PovAEouI9HR9d1Ere'
    access_token_secret  = 'tYPWl9V00DE9yM8roc6cbi2T6utMnwmRSjY1mNOj3YLbQ'

    # Sirisha credentials
#     consumer_key = 'o7q22tzgAGqQA0uugLvetKRtg'
#     consumer_secret = 'hq0GT4GuSecj0w0ewLJ2CiaRFmUmFRw8hxRwZQHm5YylslVUsz'
#     access_token = '1232466008897548294-EzoInsxXweynkssscbYJ1wBiPsQChV'
#     access_token_secret = 'Qr4VDoaLgx3AXwu4Or0tersCkFimWwY71YFzQxdhCslo6'

    # Use the credentials for authentication to the API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth,wait_on_rate_limit=True)

    # Check for the API connection, is successful or not
    if (not api):
        print("Can't Authenticate")
        exit(0)
    else:
        print("Authenticated successfully")
        return api

In [53]:
def download_tweets(api):
    ''' This function fetches the tweets and writes to a file'''
    # Open/Create an output file to append tweets
    csvFile = open(output_file, 'a')

    # Initialize the dataframe to load into
    tweets_by_region_df = pd.DataFrame(columns = ['id','text', 'language', 'created_at', 'month', 'day', 'source', 'region'])
    date = datetime.datetime(2019,11,1,0,0,0)

    # Initialize the vtracking variables
    minid = -1
    result = True

    # Count of the tweets extracted so far
    cnt = 0
    # Timeout count 
    TIME_OUT = 0

    # Iterate until we get the desired size
    while cnt < maxTweets:
        # While we still have tweets from the previous round
        while result and cnt < maxTweets:
            # For the first API call
            if minid < 0:
                print("first time")
                result = api.search(q='#covid19'+ " -filter:retweets",
                                    count=tweetCount,
                                    lang="en",
                                    tweet_mode='extended'
                                    )

            # For the subsequent API calls, get the remaining tweets apart from the ones we already  got
            else:
                print("Next onwards")
                result = api.search(q='#covid19'+ " -filter:retweets",
                                    count=tweetCount,
                                    lang = "en",
                                    tweet_mode='extended',
                                    max_id = str(minid-1)
                                    )

            # To get the full text of a tweet which are continued in other tweets           
            # Print and save results to a file
            if result:
                tweets_data = [[tweet_info.id,tweet_info.full_text, tweet_info.metadata["iso_language_code"], tweet_info.created_at] for tweet_info in result]
                tweet_df = pd.DataFrame(data=tweets_data, columns=['id','text', "language", "created_at"])

                tweet_df["month"] = tweet_df['created_at'].map(lambda x: x.month)
                tweet_df["day"] = tweet_df['created_at'].map(lambda x: x.day)
                tweet_df["source"] = "twitter"
                tweet_df["region"] = area
                tweets_by_region_df = pd.concat([tweet_df,tweets_by_region_df], ignore_index=True)

                
                tweet_df.to_csv(csvFile, index = False, header = False)
            

                # Update the count of extracted tweets
                cnt += len(result)
                print("downloaded tweets:", cnt)

                # Update the max and min id of the tweets made in this API call
                maxid = result[0].id
                minid = result[-1].id
                
                print("cnt", cnt)
                print("minid", minid)
                
                config['DEFAULT'] = {'output_file_csv': output_file,
                                    'tweetCount': tweetCount,
                                    'maxTweets':maxTweets-cnt,
                                    'minid':minid,
                                    'area':area}
        
                with open(config_file, 'w') as cf:
                    config.write(cf)
                cf.close()

                # Count the number of API calls made
                TIME_OUT += 1

                # Sleep if the number of API calls made are equal to 150
                if TIME_OUT % 150 == 0:
                    print("Completed API calls: ", TIME_OUT)
                    print("Sleeping for 15mins before making next API call.....")
                    time.sleep(60*15)
            else:
                print("Didn't fetch any tweets")
                
    return (cnt, minid-1, tweets_by_region_df)


In [54]:
######### Call the above functions in order #########
sanity_checks() 
api = authenticate()
download_count, last_min_id, downloads_df = download_tweets(api)

Sanity checks passed..
Authenticated successfully
first time
downloaded tweets: 100
cnt 100
minid 1247661309619048453
Next onwards
downloaded tweets: 200
cnt 200
minid 1247661154190733312
Next onwards
downloaded tweets: 300
cnt 300
minid 1247661014444900359
Next onwards
downloaded tweets: 400
cnt 400
minid 1247660850619482115
Next onwards
downloaded tweets: 500
cnt 500
minid 1247660688971116548
Next onwards
downloaded tweets: 589
cnt 589
minid 1247660579516329985
Next onwards
downloaded tweets: 679
cnt 679
minid 1247660477355888646
Next onwards
downloaded tweets: 779
cnt 779
minid 1247660413455646721
Next onwards
downloaded tweets: 879
cnt 879
minid 1247660235302621185
Next onwards
downloaded tweets: 979
cnt 979
minid 1247660041404141573
Next onwards
downloaded tweets: 1079
cnt 1079
minid 1247659849590202369
Next onwards
downloaded tweets: 1179
cnt 1179
minid 1247659698293219328
Next onwards
downloaded tweets: 1279
cnt 1279
minid 1247659537546575872
Next onwards
downloaded tweets: 1379

In [14]:
downloads_df['text'][0]

'#Coronavirus: Harsh Vardhan warns black marketeers against hoarding of masks \n#coronavirusindia #CoronaVirusUpdate \nhttps://t.co/RmDSO7zfHT'